In [1]:
import json
import os
import numpy as np
import yfinance as yf
import random
seed = 0
random.seed(seed)
np.random.seed(seed)
from pytickersymbols import PyTickerSymbols

def compute_qubits(stock_metadata, budget):
    n_qubits = 0
    for stock in stock_metadata:
        price = stock_metadata[stock]["price"]
        n_qubits += max(1, int(np.ceil(np.log2(np.ceil(budget/price)))))
    return n_qubits

def is_valid_ticker(ticker):
    """Check if a ticker exists by retrieving 1 day of historical data."""
    try:
        data = yf.Ticker(ticker).history(period="1d")
        return not data.empty  # Valid if data is not empty
    except:
        return False  # Invalid ticker
    
def randomly_assign_qubits(n_qubits, stocks):
    """
    Randomly assign qubits to stocks, ensuring each stock gets at least one qubit.
    
    Args:
        n_qubits: Total number of qubits
        stocks: List of stock tickers
        
    Returns:
        qubits_per_stock: Dictionary with number of qubits per stock
    """
    n_stocks = len(stocks)
    if n_qubits < n_stocks and n_qubits != n_stocks:
        raise ValueError("Number of qubits must be at least equal to number of stocks")
    
    # Start by giving each stock one qubit
    qubits_per_stock = {stock: 1 for stock in stocks}
    
    # Randomly distribute remaining qubits
    remaining_qubits = n_qubits - n_stocks
    
    # Use random.choices with weights to distribute remaining qubits
    for _ in range(remaining_qubits):
        # Randomly select a stock to give an additional qubit
        chosen_stock = np.random.choice(stocks)
        qubits_per_stock[chosen_stock] += 1
    
    return qubits_per_stock

def main():
    stock_data = PyTickerSymbols()
    stocks = list(stock_data.get_dow_jones_nyc_yahoo_tickers())
    stocks = [s for s in stocks if is_valid_ticker(s)]
    print("Available stocks: ", stocks)

    experiment_data = []
    if os.path.exists("experiments_data.json"):
        with open("experiments_data.json", "r") as f:
            experiment_data = json.load(f)["data"]

    print("Loaded previous experiment data", len(experiment_data))

    qubits_dict = {}
    for data_point in experiment_data:
        qubits = data_point["max_qubits"]
        if qubits not in qubits_dict:
            qubits_dict[qubits] = []
        qubits_dict[qubits].append(data_point)

    while True:
        for n_stocks in range(2, 11):

            for n_qubit in qubits_dict:
                finished = [len(qubits_dict[n_qubit]) > 9 for n_qubit in qubits_dict]
                if all(finished) and len(qubits_dict) == 10:
                    experiment_data = []
                    for qubits in qubits_dict:
                        for data_point in qubits_dict[qubits]:
                            experiment_data.append(data_point)
                    print("All experiments finished")
                    with open("experiments_data.json", "w") as f:
                        json.dump({"data": experiment_data}, f)
                    
                    return None

            sampled_stocks = random.sample(stocks, n_stocks)

            start = "2015-01-01"
            end = "2025-01-01"
            data = yf.download(sampled_stocks, start=start, end=end, progress=False)
            prices_now = data["Close"].iloc[-1]

            budget = random.randint(int(np.ceil(max(prices_now))), 6000)

            max_qubits = compute_qubits({stock: {"price": prices_now[stock]} for stock in sampled_stocks}, budget)
            
            if max_qubits < 16 and max_qubits > 5 and max_qubits not in qubits_dict:
                qubits_dict[max_qubits] = []

            if max_qubits > 15 or max_qubits < 6 or len(qubits_dict[max_qubits]) > 9:
                continue

            print("Budget:", budget, " Max qubits:", max_qubits, " Stocks:", sampled_stocks)
            
            data_point = {}
            data_point["max_qubits"] = max_qubits
            data_point["budget"] = budget
            data_point["stocks"] = sampled_stocks
            data_point["start"] = start
            data_point["end"] = end
            data_point["n_stocks"] = n_stocks

            qubits_dict[max_qubits].append(data_point)

In [2]:
main()

$GS-PK: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")


Available stocks:  ['MMM', 'AXP', 'AAPL', 'BA', 'CAT', 'CVX', 'CSCO', 'KO', 'GS', 'HD', 'IBM', 'INTC', 'JNJ', 'JPM', 'MCD', 'NKE', 'MRK', 'MSFT', 'PG', 'TRV', 'UNH', 'VZ', 'V', 'WMT', 'WBA', 'DIS', 'AMGN', 'HON', 'CRM', 'DOW']
Loaded previous experiment data 0
YF.download() has changed argument auto_adjust default to True
Budget: 3670  Max qubits: 10  Stocks: ['HON', 'JNJ']
Budget: 4550  Max qubits: 13  Stocks: ['AXP', 'GS', 'MRK']
Budget: 2969  Max qubits: 10  Stocks: ['TRV', 'NKE']
Budget: 2499  Max qubits: 8  Stocks: ['GS', 'MRK']
Budget: 1209  Max qubits: 9  Stocks: ['INTC', 'AMGN']
Budget: 1891  Max qubits: 10  Stocks: ['AXP', 'TRV', 'MMM']
Budget: 483  Max qubits: 15  Stocks: ['WMT', 'DIS', 'AXP', 'DOW', 'VZ']
Budget: 5710  Max qubits: 13  Stocks: ['MMM', 'NKE']
Budget: 4501  Max qubits: 14  Stocks: ['JPM', 'CRM', 'PG']
Budget: 671  Max qubits: 10  Stocks: ['AMGN', 'VZ', 'GS', 'CAT', 'MSFT']
Budget: 4964  Max qubits: 9  Stocks: ['HON', 'V']
Budget: 622  Max qubits: 13  Stocks: ['

In [4]:
experiment_data = []
if os.path.exists("experiments_data.json"):
    with open("experiments_data.json", "r") as f:
        experiment_data = json.load(f)["data"]

print(len(experiment_data), " experiments generated")
# Group by qubits and print stats
qubits_dict = {}
for data_point in experiment_data:
    qubits = data_point["max_qubits"]
    if qubits not in qubits_dict:
        qubits_dict[qubits] = []
    qubits_dict[qubits].append(data_point)
for qubits, data_points in qubits_dict.items():
    print(f"Qubits: {qubits}, Number of experiments: {len(data_points)}")

100  experiments generated
Qubits: 10, Number of experiments: 10
Qubits: 13, Number of experiments: 10
Qubits: 8, Number of experiments: 10
Qubits: 9, Number of experiments: 10
Qubits: 15, Number of experiments: 10
Qubits: 14, Number of experiments: 10
Qubits: 12, Number of experiments: 10
Qubits: 11, Number of experiments: 10
Qubits: 7, Number of experiments: 10
Qubits: 6, Number of experiments: 10
